In [ ]:
from netCDF4           import Dataset
from scipy.io          import netcdf
from scipy.integrate   import simps
from numpy             import roll

import numpy as np
import json
import matplotlib.pyplot as plt
from boutdata import collect

from Analysis import Analyse
from matplotlib.pyplot        import axis, savefig, pcolormesh, imshow, show, figure, subplot, title, plot, colorbar

In [ ]:
import os

print(os.system('ls'))

import subprocess as sp

In [ ]:
print(sp.check_output(['tail', '-n', '6', 'Analysis.py']))

In [ ]:
File_name  = '../FELTOR/output_1P.nc'
File_check = '../FELTOR/output_1P_check.nc'

Data       = Dataset(File_name)
print(Data)
Data_check = Dataset(File_check)
print(Data_check)

# plt.imshow(nc['Temp'][1,:,0,:])
# plt.show()
feltor_input = json.loads(Data.inputfile)

plot(Data['energy_time'], Data['ions_center'])
plot(Data_check['energy_time'], Data_check['ions_center'])
show()
print(np.shape(Data['ions_center']))
print(np.array_equal(Data, Data_check))

# print(np.where(Data_check['ions'] == Data['ions_center']))

In [ ]:
for key in Data_check.keys:
    print(key)

In [ ]:
# File_name_HW = '../FELTOR/output_HW.nc'
# File_name_HW_0_alpha = '../FELTOR/output_HW_0_alpha.nc'
File_name = '../FELTOR/output.nc'
# File_name_HW_00001_alpha = '../FELTOR/output_HW_00001_alpha.nc'

Data = Dataset(File_name)
print(Data)
Data = Analyse(File_name)

# plt.imshow(nc['Temp'][1,:,0,:])
# plt.show()
# feltor_input = json.loads(Data.inputfile)
np.shape(Data.Data['ions_points'])

In [ ]:
# print(nc['ions'][1, :, :])


## Plot with the grid
imshow(Data['vorticity'][1, :, :]);
show()

imshow(Data['ions'][1, :, :]);
show()

In [ ]:
# X_mat = tile(Analitics.x.reshape(Analitics.Nx, 1), Analitics.Ny)
# Y_mat = tile(Analitics.y.reshape(Analitics.Ny, 1), Analitics.Nx).transpose()


# imshow(X_mat)

## Center of Mass

In [ ]:
from numpy import tile, arange, array, copy

In [ ]:
x    = array(Data['x'][:])
Nx   = len(x)

y    = array(Data['y'][:])
Ny   = len(y)
particles = array(Data['ions'][:])


def CM(x, y, n):
    
    x_vec, y_vec = copy(x), copy(y)
    
    Nx, Ny = len(x_vec), len(y_vec)
    
    X_mat = tile(x_vec.reshape(Nx, 1), Ny)
    Y_mat = tile(y_vec.reshape(Ny, 1), Nx).transpose()
        
    X_CM, Y_CM   = np.sum(X_mat * n, axis = (1,2)), np.sum(Y_mat * n, axis = (1,2))
    n_total = np.sum(n, axis = (1,2))
    
    return X_CM / n_total, Y_CM / n_total

X_CM, Y_CM = CM(x, y, particles)


### Velocity

In [ ]:
def V_CM(X, Y, t):
    
    D_x, D_y, D_t = X - roll(X, 1), Y - roll(Y, 1), t - roll(t, 1)
    V_x, V_y      = D_x / D_t, D_y / D_t
    V_x[0] = V_y[0] = 0
    
    return V_x, V_y

V_CM_x, V_CM_y = V_CM(X_CM, Y_CM, array(Data['time'][:]))

## Integrate

In [ ]:
from scipy.integrate   import simps

In [ ]:
# print(Analitics.Data['mass'][:])
# print(Analitics.integrate('ions'))

In [ ]:

x    = array(Data['x'][:])
Nx   = len(x)

y    = array(Data['y'][:])
Ny   = len(y)
mass = array(Data['ions'][:])

Integral = simps(mass, x, axis = 1)
Integral = simps(Integral, y, axis = 1)

# print(Integral)

## Analysis class

In [ ]:
Analitics = Analyse(File_name)
Mass = Analitics.integrate('ions')
Potential = Analitics.integrate('potential')

In [ ]:
Analitics.plot_parameter(Analitics.Y_CM, Analitics.X_CM)

Analitics.plot_parameter(Mass)

In [ ]:
ftsz_title = 17
ftsz_label = 15

point = 500
X, Y = np.meshgrid(Analitics.x, Analitics.y)

figure(figsize = (24, 16))

subplot(2, 3, 1);
plot(Analitics.X_CM, Analitics.Y_CM, 'o'); title('Center of mass', fontsize = ftsz_title)
plot(Analitics.X_CM[point], Analitics.Y_CM[point], 'o', color = 'red')
plt.xlabel('CM X axis', fontsize = ftsz_label)
plt.ylabel('CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 2); 
plot(Analitics.time, (Mass[0] - Mass) / Mass[0]); title('Error in total Mass', fontsize = ftsz_title)
plot(Analitics.time[point], (Mass[0] - Mass[point]) / Mass[0], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Error in total Mass', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 3); 
plot(Analitics.V_CM_x, Analitics.V_CM_y, 'o'); title('Velocity of the CM', fontsize = ftsz_title)
plot(Analitics.V_CM_x[point], Analitics.V_CM_y[point], 'o', color = 'red')
plt.xlabel('V_CM X axis', fontsize = ftsz_label)
plt.ylabel('V_CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 4)
plot(Analitics.time, Potential); title('Total Potential', fontsize = ftsz_title)
plot(Analitics.time[point], Potential[point], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Total Potential', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 5)
pcolormesh(Analitics.x, Analitics.y, Analitics.ions[point], cmap = 'plasma', shading = 'gouraud'); title('Ions density', fontsize = ftsz_title)

subplot(2, 3, 6)
pcolormesh(Analitics.x, Analitics.y, Analitics.Data['vorticity'][point], cmap = 'plasma', shading = 'gouraud'); title('Vorticity', fontsize = ftsz_title)

show()

In [ ]:
Analitics_HW = Analyse(File_name_HW_0_alpha)
Mass_HW = Analitics_HW.integrate('ions')
Potential_HW = Analitics_HW.integrate('potential')

In [ ]:
point = 500
X, Y = np.meshgrid(Analitics_HW.x, Analitics_HW.y)

figure(figsize = (24, 16))

subplot(2, 3, 1);
plot(Analitics_HW.X_CM, Analitics_HW.Y_CM, 'o'); title(r'Center of mass HW $\alpha$ = 0', fontsize = ftsz_title)
plot(Analitics_HW.X_CM[point], Analitics_HW.Y_CM[point], 'o', color = 'red')
plt.xlabel('CM X axis', fontsize = ftsz_label)
plt.ylabel('CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 2); 
plot(Analitics_HW.time, (Mass_HW[0] - Mass_HW) / Mass_HW[0]); title(r'Error in Total Mass HW $\alpha$ = 0', fontsize = ftsz_title)
plot(Analitics_HW.time[point], (Mass_HW[0] - Mass_HW[point]) / Mass_HW[0], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Error in Total Mass', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 3); 
plot(Analitics_HW.V_CM_x, Analitics_HW.V_CM_y, 'o'); title(r'Velocity of the CM HW $\alpha$ = 0', fontsize = ftsz_title)
plot(Analitics_HW.V_CM_x[point], Analitics_HW.V_CM_y[point], 'o', color = 'red')
plt.xlabel('V_CM X axis', fontsize = ftsz_label)
plt.ylabel('V_CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 4)
plot(Analitics_HW.time, Potential_HW); title(r'Total Potential HW $\alpha$ = 0', fontsize = ftsz_title)
plot(Analitics_HW.time[point], Potential_HW[point], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Total Potential', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 5)
pcolormesh(Analitics_HW.x, Analitics_HW.y, Analitics_HW.ions[point], cmap = 'plasma', shading = 'gouraud'); title(r'Ions density HW$\alpha$ = 0', fontsize = ftsz_title)

subplot(2, 3, 6)
pcolormesh(Analitics_HW.x, Analitics_HW.y, Analitics_HW.Data['vorticity'][point], cmap = 'plasma', shading = 'gouraud'); title(r'Vorticity HW $\alpha$ = 0', fontsize = ftsz_title)

show()

In [ ]:
Analitics_HW = Analyse(File_name_HW_00001_alpha)
Mass_HW = Analitics_HW.integrate('ions')
Potential_HW = Analitics_HW.integrate('potential')

In [ ]:
point = 300
X, Y = np.meshgrid(Analitics_HW.x, Analitics_HW.y)

figure(figsize = (24, 16))

subplot(2, 3, 1);
plot(Analitics_HW.X_CM, Analitics_HW.Y_CM, 'o'); title(r'Center of mass HW $\alpha$ = 0.00001', fontsize = ftsz_title)
plot(Analitics_HW.X_CM[point], Analitics_HW.Y_CM[point], 'o', color = 'red')
plt.xlabel('CM X axis', fontsize = ftsz_label)
plt.ylabel('CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 2); 
plot(Analitics_HW.time, (Mass_HW[0] - Mass_HW) / Mass_HW[0]); title(r'Error in Total Mass HW $\alpha$ = 0.00001', fontsize = ftsz_title)
plot(Analitics_HW.time[point], (Mass_HW[0] - Mass_HW[point]) / Mass_HW[0], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Error in Total Mass', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 3); 
plot(Analitics_HW.V_CM_x, Analitics_HW.V_CM_y, 'o'); title(r'Velocity of the CM HW $\alpha$ = 0.00001', fontsize = ftsz_title)
plot(Analitics_HW.V_CM_x[point], Analitics_HW.V_CM_y[point], 'o', color = 'red')
plt.xlabel('V_CM X axis', fontsize = ftsz_label)
plt.ylabel('V_CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 4)
plot(Analitics_HW.time, Potential_HW); title(r'Total Potential HW $\alpha$ = 0.00001', fontsize = ftsz_title)
plot(Analitics_HW.time[point], Potential_HW[point], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Total Potential', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 5)
pcolormesh(Analitics_HW.x, Analitics_HW.y, Analitics_HW.ions[point], cmap = 'plasma', shading = 'gouraud'); title(r'Ions density HW$\alpha$ = 0.00001', fontsize = ftsz_title)

subplot(2, 3, 6)
pcolormesh(Analitics_HW.x, Analitics_HW.y, Analitics_HW.Data['vorticity'][point], cmap = 'plasma', shading = 'gouraud'); title(r'Vorticity HW $\alpha$ = 0.00001', fontsize = ftsz_title)

show()

In [ ]:
Analitics_HW = Analyse(File_name_HW)
Mass_HW = Analitics_HW.integrate('ions')
Potential_HW = Analitics_HW.integrate('potential')

In [ ]:
point = 500
X, Y = np.meshgrid(Analitics_HW.x, Analitics_HW.y)

figure(figsize = (24, 16))

subplot(2, 3, 1);
plot(Analitics_HW.X_CM, Analitics_HW.Y_CM, 'o'); title(r'Center of mass HW $\alpha$ = 0.01', fontsize = ftsz_title)
plot(Analitics_HW.X_CM[point], Analitics_HW.Y_CM[point], 'o', color = 'red')
plt.xlabel('CM X axis', fontsize = ftsz_label)
plt.ylabel('CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 2); 
plot(Analitics_HW.time, (Mass_HW[0] - Mass_HW) / Mass_HW[0]); title(r'Error in Total Mass HW $\alpha$ = 0.01', fontsize = ftsz_title)
plot(Analitics_HW.time[point], (Mass_HW[0] - Mass_HW[point]) / Mass_HW[0], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Error in Total Mass', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 3); 
plot(Analitics_HW.V_CM_x, Analitics_HW.V_CM_y, 'o'); title(r'Velocity of the CM HW $\alpha$ = 0.01', fontsize = ftsz_title)
plot(Analitics_HW.V_CM_x[point], Analitics_HW.V_CM_y[point], 'o', color = 'red')
plt.xlabel('V_CM X axis', fontsize = ftsz_label)
plt.ylabel('V_CM Y axis', fontsize = ftsz_label)


subplot(2, 3, 4)
plot(Analitics_HW.time, Potential_HW); title(r'Total Potential HW $\alpha$ = 0.01', fontsize = ftsz_title)
plot(Analitics_HW.time[point], Potential_HW[point], 'o', color = 'red')
plt.xlabel('Time', fontsize = ftsz_label)
plt.ylabel('Total Potential', fontsize = ftsz_label)
plt.xscale('log')


subplot(2, 3, 5)
pcolormesh(Analitics_HW.x, Analitics_HW.y, Analitics_HW.ions[point], cmap = 'plasma', shading = 'gouraud'); title(r'Ions density HW $\alpha$ = 0.01', fontsize = ftsz_title)

subplot(2, 3, 6)
pcolormesh(Analitics_HW.x, Analitics_HW.y, Analitics_HW.Data['vorticity'][point], cmap = 'plasma', shading = 'gouraud'); title(r'Vorticity HW $\alpha$ = 0.01', fontsize = ftsz_title)

show()

# Correlation

The Next thing we wanna do is to make the spacial and temporal correlation between Potential and density. For that, first we need to define the cross correlation:

$$ (f * g)(\tau) = \int_{-\infty}^{\infty}\overline{f}( t) g( t + \tau) dt$$

which in the discrete form gets:

$$ (f * g)(n) = \sum_{m = 0} ^{M}\overline{f}(m) g(m + n)$$


In [ ]:
from scipy.signal    import correlate2d
from numpy           import average, shape, copy, correlate, vectorize, sqrt, array
from numpy           import transpose, copy, empty
from scipy.integrate import simps

from time  import time

import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

from Analysis import Analyse
from matplotlib.pyplot        import axis, savefig, pcolormesh, imshow, show, figure, subplot, title, plot, colorbar

File      = '../marconi_outputs/hdiff_outputs/3_tpts/output_IC.nc'
Analitics = Analyse(File)

In [ ]:
File      = '../marconi_outputs/hdiff_outputs/3_tpts/output_IC.nc'
Analitics = Analyse(File)

ions      = copy(Analitics.ions)
potential = copy(Analitics.potential)



In [ ]:
(Analitics.time - roll(Analitics.time, -1))[-1:]

In [ ]:
start = time()

x_cp, y_cp = copy(ions), copy(potential)
    
lnx, mx, nx = shape(x_cp)
lny, my, ny = shape(y_cp)
x_vrg = average(x_cp, axis = 0).reshape(1, mx, nx)
y_vrg = average(y_cp, axis = 0).reshape(1, my, ny)

dividendo = np.sum((x_cp - x_vrg) * (y_cp - y_vrg), axis=(1, 2))
divisor = np.sum((x_cp - x_vrg) ** 2, axis=(1, 2))

divisor *= np.sum((y_cp - y_vrg) ** 2, axis=(1, 2))

correlation = dividendo / sqrt(divisor)

print(time() - start)

In [ ]:
cor = lambda x, y: array([correlate2d(x[i], y[i], 'same', 'fill') for i in range(len(x))])


def temp_correlation(x, y):
        
    x_vrg = average(x, axis = (1, 2)).reshape(len(x_cp), 1, 1)
    y_vrg = average(y, axis = (1, 2)).reshape(len(y_cp), 1, 1)
    
    dividendo = np.sum((x_cp - x_vrg) * (y_cp - y_vrg), axis=(1, 2))

    divisor  = np.sum((x_cp - x_vrg) ** 2, axis=(1, 2))
    divisor *= np.sum((y_cp - y_vrg) ** 2, axis=(1, 2))

    correlation = dividendo / sqrt(divisor)
    
def spt_correlation(x, y):
        
    x_vrg = average(x, axis = 0)
    y_vrg = average(y, axis = 0)
    
    ## Fill for DIR and NEU, WRAP for PER PER
    return correlate2d(x_vrg, y_vrg, 'same', 'fill')
    
    
def temp_cor_2 (x, y):
    
    lnx, mx, nx = shape(x)
    lny, my, ny = shape(y)
    x_vrg = average(x, axis = 0).reshape(1, mx, nx)
    y_vrg = average(y, axis = 0).reshape(1, my, ny)
    
    dividendo = np.sum((x - x_vrg) * (y - y_vrg), axis=(1, 2))

    divisor  = np.sum((x - x_vrg) ** 2, axis=(1, 2))
    divisor *= np.sum((y - y_vrg) ** 2, axis=(1, 2))

    correlation = dividendo / sqrt(divisor)
    return correlation

def spt_cor_2 (x, y):
    
#     x_vrg = average(x, axis = 0)
#     y_vrg = average(y, axis = 0)
    if dim(x) == 3:
        correlation = array([correlate2d(x[i], y[i], 'same', 'fill') for i in range(len(x))])
    
    if dim(y) == 2:
        correlate2d(x, y, 'same', 'fill')
    ## Fill for DIR and NEU, WRAP for PER PER
    return correlation

def temp_cor_2 (x, y):
    
    lnx, mx, nx = shape(x)
    lny, my, ny = shape(y)
    x_vrg = average(x, axis = 0).reshape(1, mx, nx)
    y_vrg = average(y, axis = 0).reshape(1, my, ny)
    
    dividendo = np.sum((x - x_vrg) * (y - y_vrg), axis=(1, 2))

    divisor  = np.sum((x - x_vrg) ** 2, axis=(1, 2))
    divisor *= np.sum((y - y_vrg) ** 2, axis=(1, 2))

    correlation = dividendo / sqrt(divisor)
    return correlation

def c_corr_dt (X, Y):
    '''
    cross-correlation time-delay, X and Y should be integrated
    '''
    
    dividendo = correlate(X, Y, 'same')
    divisor   = correlate(X, X)
    divisor  *= correlate(Y, Y)
    
    return dividendo / sqrt(divisor)
    
    
def c_corr_sp (f, g, x0, integrate_y = False):
    '''
    Spatial cross-correlation, x and y should not be integrated
    '''
    
    fcg = f[:, x0] * g[:, x0]
    
    
    fcg_vrg = average(fcg, axis = 0)
    
    if integrate_y:
        fcg_vrg = average(y, axis = 0)
    
    return correlate2D(x_vrg, y_vrg)

def c_corr_dt (self, f, g, time_units = 1):
    '''
    cross-correlation time-delay
    '''

    assert array_equal(shape(f), shape(g)), 'The dimensions of f and g should be equal'

    nt, nx, ny = shape(f)
    steps = int(time_units / (self.time[1] - self.time[0]))
    corr = empty((nt * steps + 1, nx, ny))
    
    for ix in range(nx):
        for iy in range(ny):
            corr [:, ix, iy] = CC(f[:, ix, iy], g[:, ix, iy], time_units)

    corr = self.integrate(corr, 1, axis = 2) /  (self.y[-1] - self.y[0])

    return corr
    
def CC (self, f, g, time_units):
    
    steps = int(time_units / (self.time[1] - self.time[0]))

    corr  = empty(2 * steps + 1)
    for i in range(-steps, steps + 1):
        corr[i] = np.sum(f * np.roll(g, i))
    return corr
    

In [ ]:
steps = int(1.1 / 0.25)
a = np.empty((2 * steps + 1, 23))
for i in range( -steps, steps + 1):
    a[i] = np.zeros(23)
print(a)

In [ ]:
nx = None
ny = None

%timeit shape(np.zeros((100, 20, 100)))
%timeit  nx == ny == None

In [ ]:
def c_corr_sp (f, g, x0=None, y0=None, axis = 1):
    '''
    Spatial cross-correlation, x and y should not be integrated
    '''
    
    
    if not np.array_equal(shape(f), shape(g)):
        raise Exception('The dimensions of f and g should be equal')
        
    if type(x0) == type(None) and type(y0) != type(None):
        f_cp = copy(transpose(f, (0, 2, 1)))
        g_cp = copy(transpose(g, (0, 2, 1)))
                
        z_cp = copy(y0)
        axis = 1
            
        transpose_back = True
        
    if type(x0) != type(None) and type(y0) == type(None):
        f_cp = copy(f)
        g_cp = copy(g)
                
        z_cp = copy(x0)
        axis = 0
        
        transpose_back = False
    
    else:
        raise TypeError('x0 and y0 are None')
    
    nt, nx, ny = shape(f_cp)
    mz         = shape(z_cp)
    
    if len(mz) < 1:
        mz = 1
        z_cp = [z_cp]
    else:
        mz = mz[0]
             
    fcg = empty((mz, 2 * ny + 1))
    
    ### We cannot average it like this, vrg(a) * vrg(b) != vrg(a * b)
    f_cp = average(f_cp, axis = 0)
    g_cp = average(g_cp, axis = 0)
    
    for j in range(mz):
#         auto_corr  = correlate(f_cp[z_cp[j], :], f_cp[z_cp[j], :]) 
#         auto_corr *= correlate(g_cp[z_cp[j], :], g_cp[z_cp[j], :])
        cr_corr    = correlate(f_cp[z_cp[j], :], g_cp[z_cp[j], :], 'full')
        fcg[j, :]  = cr_corr #/ sqrt(auto_corr)

#     if transpose_back:
#         fcg = np.transpose(fcg, (2, 1))

#     fcg = average(fcg, axis = 0)

    return np.squeeze(fcg)

def c_corr_dt (X, Y):
    '''
    cross-correlation time-delay
    '''
    
    x_cp = copy(X)
    y_cp = copy(Y)
        
    nt, nx, ny = shape(X)
    corr = np.empty((2 * nt - 1, nx, ny))
    
    for ix in range(nx):
        for iy in range(ny):
            dividendo = correlate(x_cp[:, ix, iy], y_cp[:, ix, iy], 'full')
#             divisor   = correlate(x_cp[:, ix, iy], x_cp[:, ix, iy])
#             divisor  *= correlate(y_cp[:, ix, iy], y_cp[:, ix, iy])
            corr [:, ix, iy] = dividendo #/ sqrt(divisor)
    
    x, y = Analitics.x, Analitics.y
    Integral = simps(corr, x, axis = 1) / (x[-1] - x[0])
    Integral = simps(Integral, y, axis = 1) / (y[-1] - y[0])

    return Integral

def correlation(x, y):
    
    x_vrg = average(x)
    y_vrg = average(y)
    
    dist_x = x - x_vrg
    dist_y = y - y_vrg
    
    
    dividendo = np.sum(dist_x * dist_y)
    divisor   = np.sum(dist_x ** 2) * np.sum(dist_y ** 2)
    
    return dividendo / sqrt(divisor)

In [ ]:
np.random.seed(9)

x = np.random.random(200)
y = np.random.random(200)

In [ ]:
print(correlation(x, y))

corr = correlate(x, y, 'full') / sqrt(correlate(x, x) * correlate(y, y))

print(np.corrcoef(x, y))

In [ ]:
corr = np.empty(200)

for i in range(200):
    corr[i] = np.sum(x * np.roll(y, i)) / sqrt( np.sum(x * np.roll(x, i)) * np.sum(y * np.roll(y, i)))
print(corr )    

## Proving new functions

In [ ]:
File      = '../marconi_outputs/hdiff_outputs/3_tpts//output_IC.nc'
Analitics = Analyse(File)

In [ ]:
tau = np.arange(-450, 451) #Analitics.time - Analitics.time[len(Analitics.time) // 2]
plt.plot(tau, Analitics.c_corr_dt(Analitics.ions, Analitics.potential))
plt.xlabel(r"$\tau$", fontsize= 15)
plt.ylabel("Time correlation", fontsize= 15)
plt.show()

In [ ]:
# tau = Analitics.time - Analitics.time[len(Analitics.time) // 2]
plt.plot(tau, c_corr_dt(Analitics.ions, Analitics.potential))
plt.xlabel(r"$\tau$", fontsize= 15)
plt.ylabel("Time correlation", fontsize= 15)
plt.show()

In [ ]:
corr = Analitics.c_corr_sp(Analitics.ions, Analitics.potential, x0=90, integrate = False)
y   = copy(Analitics.y)
tau = y - y[len(y) // 2]

plot( corr)
plt.xlabel(r"$\tau$", fontsize= 15)
plt.ylabel("Spatial correlation", fontsize= 15)
plt.show()

In [ ]:
corr = c_corr_sp(Analitics.ions[1:], Analitics.potential[1:], x0=90)
y    = copy(Analitics.y)
tau  = y - y[len(y) // 2]

plot(tau, corr)
plt.xlabel(r"$\tau$", fontsize= 15)
plt.ylabel("Spatial correlation", fontsize= 15)
plt.show()

In [ ]:
import os
import numpy as np

models = ["HW_mod"]

orig_dir   = "../FELTOR/"
new_dir    = ""
directory  = os.path.join(orig_dir, new_dir)

exists = os.path.isdir(directory)
if not exists:
    os.mkdir(directory)
else:
    print('The directory already exists')


for model in models:

    File = orig_dir + f"input.json"
    with open(File, 'r') as json_file:
            data = json.load(json_file)
    json_file.close()

    data['maxout'] = 450
    data['itstp']  = 200
    data['bc_x']   = "PER"
    data['n_out']  = 3
    data['Nx_out'] = 60
    data['Ny_out'] = 60
    data['Points'] = [2, 3, 4]
    

    with open(File, 'w') as outfile:
        json.dump(data, outfile)